## Lab 3. Fine-tuning

In [ ]:
!pip install "transformers>=4.46.0" "peft>=0.7.1" "trl==0.12.1" "accelerate>=1.0.1" "datasets>=3.1.0" "autoawq==0.2.9" bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for autoawq: filename=autoawq-0.2.9-py3-none-any.whl size=115106 sha256=01cc3f0247b0ab63f64615de769012da27363109651ade47f6dd54b15d605d71
  Stored in directory: /root/.cache/pip/wheels/45/1a/7b/7314b3a958454e8ce349f600829a3f0a6a05aeebf987be1e16
Successfully built autoawq
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import gc
import numpy as np
import random
import transformers.activations
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset

if not hasattr(transformers.activations, "PytorchGELUTanh"):
    transformers.activations.PytorchGELUTanh = transformers.activations.NewGELUActivation

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    set_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

gc.collect()
torch.cuda.empty_cache()
seed_everything(42)  

MODEL_ID = "NOVORDSEC/qwen3-8b-awq-int4"
NEW_MODEL_NAME = "qwen3-8b-awq-finetuned"
DATASET_NAME = "mlabonne/guanaco-llama2-1k"

def train():
    print(f"GPUs available: {torch.cuda.device_count()}")
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    print("Loading model...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto", 
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    model.config.use_cache = False 
    model.gradient_checkpointing_enable() 
    model = prepare_model_for_kbit_training(model)
    model.enable_input_require_grads() 

    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=8,            
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "v_proj"] 
    )
    
    model = get_peft_model(model, peft_config)
    print("\nTrainable parameters:")
    model.print_trainable_parameters()

    dataset = load_dataset(DATASET_NAME, split="train")

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=1,   
        gradient_accumulation_steps=4,   
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=True,
        bf16=False,
        max_grad_norm=0.3,
        logging_steps=10,
        optim="paged_adamw_32bit",       
        lr_scheduler_type="constant",
        save_strategy="no",
        report_to="none",
        ddp_find_unused_parameters=False,
        gradient_checkpointing=False,
        seed=42,          
        data_seed=42      
    )

    print("\nStarting training...")
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=256, 
        tokenizer=tokenizer,
        args=training_args,
        packing=False,
    )

    trainer.train()

    print(f"\nSaving model to {NEW_MODEL_NAME}...")
    trainer.model.save_pretrained(NEW_MODEL_NAME)
    tokenizer.save_pretrained(NEW_MODEL_NAME)
    print("Training completed successfully.")

if __name__ == "__main__":
    train()

GPUs available: 2
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]


Trainable parameters:
trainable params: 3,833,856 || all params: 1,248,801,792 || trainable%: 0.3070


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Step,Training Loss
10,1.645600
20,1.505700
30,1.299000
40,1.335400
50,1.302500
60,1.418800
70,1.355900
80,1.375900
90,1.469900
100,1.331600


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



Saving model to qwen3-8b-awq-finetuned...
Training completed successfully.


In [ ]:
import torch
import gc
import os
import pandas as pd
import numpy as np
import random
import transformers.activations
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from peft import PeftModel
from datasets import load_dataset
from huggingface_hub import model_info

if not hasattr(transformers.activations, "PytorchGELUTanh"):
    transformers.activations.PytorchGELUTanh = transformers.activations.NewGELUActivation

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    set_seed(seed)

gc.collect()
torch.cuda.empty_cache()
seed_everything(42)

BASE_MODEL_ID = "NOVORDSEC/qwen3-8b-awq-int4" 
ADAPTER_REPO_ID = "NOVORDSEC/qwen3-8b-awq-finetuned-adapters"
ORIG_MODEL_ID = "Qwen/Qwen3-8B"
ORIG_ACC = 0.7292 #Получен на первом этапе, здесь просто неохота было второй раз считать
FRACTION = 0.2

def get_remote_size_gb(repo_id):
    try:
        info = model_info(repo_id, files_metadata=True)
        size = sum(s.size for s in info.siblings if s.size and any(ext in s.rfilename for ext in ['.safetensors', '.bin', '.pt']))
        return size / (1024**3)
    except Exception as e:
        print(f"Error getting size for {repo_id}: {e}")
        return 0

def run_mmlu_benchmark(model, tokenizer, fraction=0.2):
    subjects = [
        'abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 
        'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 
        'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 
        'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 
        'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 
        'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 
        'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 
        'high_school_physics', 'high_school_psychology', 'high_school_statistics', 
        'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 
        'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 
        'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 
        'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 
        'professional_law', 'professional_medicine', 'professional_psychology', 
        'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions'
    ]
    
    device = model.device
    choices = ["A", "B", "C", "D"]
    choice_tokens = [tokenizer.encode(f" {c}", add_special_tokens=False)[-1] for c in choices]
    
    detailed_results = {}
    total_correct = 0
    total_questions = 0
    
    model.eval()
    
    for subject in tqdm(subjects, desc="MMLU Inference"):
        try:
            dataset = load_dataset("cais/mmlu", subject, split="test")
            num_samples = max(1, int(len(dataset) * fraction))
            dataset = dataset.select(range(num_samples))
            
            sub_correct = 0
            for item in dataset:
                prompt = f"The following are multiple choice questions (with answers) about {subject.replace('_', ' ')}.\n\n"
                prompt += f"{item['question']}\n"
                prompt += f"(A) {item['choices'][0]}\n(B) {item['choices'][1]}\n(C) {item['choices'][2]}\n(D) {item['choices'][3]}\n"
                prompt += "Answer:"
                
                inputs = tokenizer(prompt, return_tensors="pt").to(device)
                
                with torch.inference_mode():
                    logits = model(**inputs).logits[0, -1, :]
                    relevant_logits = logits[choice_tokens]
                    pred = torch.argmax(relevant_logits).item()
                    
                    if pred == item['answer']:
                        sub_correct += 1
            
            acc = sub_correct / num_samples
            detailed_results[subject] = acc
            total_correct += sub_correct
            total_questions += num_samples
        except Exception as e:
            print(f"Error in {subject}: {e}")
            
    return (total_correct / total_questions if total_questions > 0 else 0), detailed_results

if __name__ == "__main__":
    print("Calculating model sizes...")
    size_orig = get_remote_size_gb(ORIG_MODEL_ID)
    size_base = get_remote_size_gb(BASE_MODEL_ID)
    size_adapter = get_remote_size_gb(ADAPTER_REPO_ID)

    final_size = size_base + size_adapter
    ratio = size_orig / final_size if final_size > 0 else 0

    print(f"Original Size: {size_orig:.2f} GB")
    print(f"Base Size: {size_base:.2f} GB")
    print(f"Adapter Size: {size_adapter:.4f} GB")
    print(f"Total Final Size: {final_size:.2f} GB")
    print(f"Compression Ratio: {ratio:.2f}x")

    print("\nLoading Base Model...")
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )

    print("Loading Adapters...")
    model = PeftModel.from_pretrained(model, ADAPTER_REPO_ID)
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)

    print("\nRunning Benchmark...")
    final_acc, detailed_results = run_mmlu_benchmark(model, tokenizer, fraction=FRACTION)

    drop = (ORIG_ACC - final_acc) / ORIG_ACC
    score = ratio / (1 + drop)

    print("\n" + "="*40)
    print(f"ИТОГИ ЭТАПА 2 (Fine-Tuning):")
    print(f"Compression Ratio: {ratio:.4f}")
    print(f"Original Accuracy: {ORIG_ACC:.4f}")
    print(f"Final Accuracy: {final_acc:.4f}")
    print(f"Performance Drop: {drop*100:.2f}%")
    print(f"ФИНАЛЬНЫЙ SCORE: {score:.4f}")
    print("="*40)

    subjects = sorted(list(detailed_results.keys()))
    csv_data = []
    for s in subjects:
        csv_data.append({
            "Subject": s,
            "Finetuned_Acc": detailed_results.get(s, 0.0)
        })

    df_final = pd.DataFrame(csv_data)
    df_final.to_csv("mmlu_finetuned_detailed.csv", index=False)
    print(f"Детальный отчет сохранен в mmlu_finetuned_detailed.csv")

Calculating model sizes...
Original Size: 15.26 GB
Base Size: 5.68 GB
Adapter Size: 0.0143 GB
Total Final Size: 5.69 GB
Compression Ratio: 2.68x

Loading Base Model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Adapters...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/171 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

MMLU Inference: 100%|██████████| 57/57 [35:48<00:00, 37.69s/it]


ИТОГИ ЭТАПА 2 (Fine-Tuning):
Compression Ratio: 2.6793
Original Accuracy: 0.7292
Final Accuracy: 0.7210
Performance Drop: 1.13%
ФИНАЛЬНЫЙ SCORE: 2.6495
Детальный отчет сохранен в mmlu_finetuned_detailed.csv
